# Stream Ciphers

In [1]:
import streamcipher
from streamcipher import LFSR
from streamcipher import berlekamp_massey
from streamcipher import ShrinkingGenerator
from functools import reduce
from operator import xor

## Introduction

**Stream Ciphers** are symmetric key algorithms that encrypt the plaintext combining it with the **keystream**, a sequence of bits obtained from the key, typically applying a bitwise XOR operation. Due to the fact that bits can be seen as elements of GF(2), the inverse of the XOR operation is the XOR operation itself, then decryption is achieved by applying the same procedure to the ciphertext.[[1]](#references)

To avoid to have a key that is as long as the plaintext/ciphertext, the keystream is typically produced using a **Pseudorandom Number Generators (PRNG)**, commonly based on **Linear Feedback Shift Registers (LFSR)**.

![Stream Cipher general structure](../Report%20Images/stream%20cipher.jpg)

In this notebook one will see the implementation of an LFSR together with the application of the Berlekamp-Massey algorithm to break it. Eventually a Shrinking Generator will be constructed.

## LFSR

A **Linear Feedback Shift Register (LFSR)** is a widely used component in cryptographic systems to generate pseudorandom binary sequences. It consists of a shift register whose input bit is determined by a linear combination of its previous state: this combination is performed using a **feedback function**, typically encoded as a polynomial, that selects certain state bits to be XORed together and fed back into the register. The output sequence is always **periodic** and its period depends only on the structure of the LFSR. As different LFSRs can produce identical sequences, the optimal design prioritizes the shortest register length in order to reduce power consumption and area. [[2]](#references)

![Linear Feedback Shift Regsiter general structure](../Report%20Images/lfsr.jpg)

### LFSR implementation

An LFSR with feedback polynomial `[4, 1, 0]` is defined to test the functionalities of the class. The latter is characterized by:
- Attributes:
    - *poly*: feedback polynomial
    - *state*: current state
    - *output*: current output bit value
    - *feedback*: current feedback bit value
- Methods:
    - *new_iteration*: compute a new_iteration of the LFSR, updating state and output
    - *\_\_iter\_\_* and *\_\_next\_\_*: set up the iterator
    - *run_steps*: compute the output sequence corresponding to the next N iterations
    - *cycle*: returns the output sequence constituting a period of the LFSR
    - *\_\_str\_\_*: provides a description of the LFSR (feedback polynomial, LFSR length and current state)

In [2]:
lfsr = LFSR([4, 1, 0])
# Check whether the LFSR was properly set
lfsr.__str__()
# Compute a period
print(lfsr.cycle())
# Compute 6 iterations
print(lfsr.run_steps(6))
# See how the internal state of the LFSR has changed
lfsr.__str__()
# Check the iterator functionality
for n, b in enumerate(lfsr):
    if n == 10:
        break
    print(b)
# Check the LFSR attributes
print(f'LFSR polynomial: {lfsr.poly}, \
    \nLFSR current state: {lfsr.state}, \
    \nLFSR current output bit: {lfsr.output}, \
    \nLFSR current feedback bit: {lfsr.feedback}')



Feedback polynomial: [0, 1, 4]
LFSR length: 4            
Current state: b'\x0f'

Cycle completed. 15 elements
[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]

6 iterations of the LFSR have been computed
[1, 1, 1, 1, 0, 1]

Feedback polynomial: [0, 1, 4]
LFSR length: 4            
Current state: b'\x06'
0
1
1
0
0
1
0
0
0
1
LFSR polynomial: [0, 1, 4],     
LFSR current state: b'\x0b',     
LFSR current output bit: 1,     
LFSR current feedback bit: 1


## Berlekamp Massey

Despite appearing to have good properties, both in terms of output sequence randomicity and of hardware implementation, the LFSR cannot be used as is for random number generation in cryptosystems because of the existance of the **Berlekamp-Massey algorithm**: the latter, given enough keystream bits, can provide the shortest LFSR able to produce the LFSR output sequence. [[3]](#references)

During a **Known Plaintext Attack**, the eavesdropper can compute the keystream bits from the observed plaintext and ciphertext bits, then apply the Berlekamp-Massey algorithm to infer the LFSR feedback polynomial and eventually its initial state. At this point the attacker can compute the entire keystream and decrypt the ciphertext

### Algorithm testing

In the following step the implementation of Berlekamp-Massey Algorithm will be tested providing as input the sequences produced by the following LFSR:
- `poly = [3, 1, 0]` and `state = '\x06'`
- `poly = [5, 2, 1, 0]` and `state = '\x05'`
- `poly = [96, 81, 17, 15, 0]` and `state = b'streamcipher'`

In [3]:
lfsr = LFSR([3, 1, 0], b'\x06')
bit_sequence = lfsr.cycle()
# print(bit_sequence)
shortest_LFSR = berlekamp_massey(bit_sequence)
print(f'The shortest LFSR able to produce the provided sequence is {shortest_LFSR}')


Cycle completed. 7 elements
The shortest LFSR able to produce the provided sequence is [0, 1, 3]


In [4]:
# This polynomial is not irreducible, so we expect as output a shorter polynomial
lfsr = LFSR([1, 5, 2, 0], b'\x05')
bit_sequence = lfsr.cycle()
# print(bit_sequence)
shortest_LFSR = berlekamp_massey(bit_sequence)
print(f'The shortest LFSR able to produce the provided sequence is {shortest_LFSR}')


Cycle completed. 7 elements
The shortest LFSR able to produce the provided sequence is [0, 1, 3]


In [5]:
lfsr = LFSR([0, 17, 81, 15, 95], b'streamcipher')
# The cycle takes too long to be computed. For time reasons the provided 
# sequence is made of the minimum amount of bits hat allows to correctly guess
# the LFSR feedback polynomial
bit_sequence = lfsr.run_steps(190)
# print(bit_sequence)
shortest_LFSR = berlekamp_massey(bit_sequence)
print(f'The shortest LFSR able to produce the provided sequence is {shortest_LFSR}')


190 iterations of the LFSR have been computed
The shortest LFSR able to produce the provided sequence is [0, 15, 17, 81, 95]


Load `'binary_sequence'` (the result is a `bytes` type variable) and apply Berlekamp-Massey algorithm to find the polynomial corresponding to the shortest LFSR that can produce that sequence.

In [6]:
with open('binary_sequence.bin', 'rb') as f:
    bit_sequence = f.read()
bit_sequence = streamcipher.bytes_to_bool_list(bit_sequence)
# print(bit_sequence)
shortest_LFSR = berlekamp_massey(bit_sequence)
print(f'The shortest LFSR able to produce the provided sequence is {shortest_LFSR}')

The shortest LFSR able to produce the provided sequence is [0, 2, 9, 12, 13, 14, 16]


## LFSR-based Generators

In [7]:
# Convert the group name into a 'bytes' variable.
# If the parity bit is 0 implement a shrinking generator, otherwise
# implement a self-shrinkin generator
group_name = b'CipherCrafters'
group_name_bits = streamcipher.bytes_to_bool_list(group_name)
parity_bit = reduce(xor, group_name_bits)
parity_bit

0

### Shrinking Generator

Utilizing LFSRs as individual PRNGs is unsecure since they can be broken using Berlekamp-Massey algorithm. However, they serve as foundational components for more robust generators like the **Shrinking Generator** [[4]](#references). The latter, that belongs to the **Alternating Clocking Generator** family, is made of three key elements:

- LFSR_A: Generates potential keystream bits.
- LFSR_S: Determines whether bits from LFSR_A are retained or discarded.
- FIFO: Maintains a regular output rate by sampling LFSR_A bits only when LFSR_S produces a 1. As LFSR_S generates a pseudo-random sequence, in average half of LFSR_A's output bits are preserved while the other half are discarded. Consequently, the FIFO operates at half the frequency of the LFSRs.

#### Test the Shrinking Generator class

A shrinking Generator with the specifications written below is defined to test the functionalities of the class. The latter is characterized by:
- Attributes:
    - *lfsrA* and *lfsrS*: the two LFSRs of the generator, defined as instances of the LFSR class
    - *output*: current output bit value
- Methods:
    - *new_iteration*: computes an iteration of the generator
    - *\_\_iter\_\_* and *\_\_next\_\_*:set up the iterator
    - *run_steps*: returns the output sequence corresponding to N iterations 

In [8]:
# stateA = b'\x06', stateS = b'\x05'
# polyA = [0, 2, 5], polyS = [0, 1, 3] <--- default polynomials if not specified
sh_gen = ShrinkingGenerator(b'\x06', b'\x05')
sh_gen.lfsrA.__str__()
sh_gen.lfsrS.__str__()
# Produce 20 "valid" outputs
g_seq = sh_gen.run_steps(20)
print(g_seq)
for n, b in enumerate(sh_gen):
    if n == 5:
        break
    print(b)
print(f'Current output bit value: {sh_gen.output}')


Feedback polynomial: [0, 2, 5]
LFSR length: 5            
Current state: b'\x06'

Feedback polynomial: [0, 1, 3]
LFSR length: 3            
Current state: b'\x05'

20 iterations of the Shrinking Generator have been computed
[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0]
0
None
1
None
None
Current output bit value: 1


Use the Shrinking Generator with the following specifications to produce the keystream to decrypt the ciphertext in `ciphertext_shrinking.bin`

In [9]:
polyA = [16, 15, 12, 10, 0]
polyS = [24, 11, 5, 2, 0]
stateA = b'\xc5\xd7'
stateS = b'\x14\x84\xf8'

sh_gen = ShrinkingGenerator(stateA, stateS, polyA, polyS)

In [10]:
with open('ciphertext_shrinking.bin', 'rb') as f:
    ciphertext_shrinking = f.read()

In [11]:
ciphertext_shrinking = streamcipher.bytes_to_bool_list(ciphertext_shrinking)

# Produce as many keystream bits as ciphertext bits
sh_seq = sh_gen.run_steps(len(ciphertext_shrinking))

# Decryption consists of a bitwise XOR between ciphertext adn keystream
shrinking_plaintext = [cipher ^ gen for cipher, gen in 
                       zip(ciphertext_shrinking, sh_seq)]

shrinking_plaintext = streamcipher.bool_list_to_bytes(shrinking_plaintext)

print(shrinking_plaintext)


4992 iterations of the Shrinking Generator have been computed
b'The Shrinking Generator\nDon Coppersmith, Hugo Krawczyk, Yishay Mansour\nIBM T.J. Watson Research Center\nYorktown Heights, NY 10598\n\nAbstract. We present a new construction of a pseudorandom generator based on a simple combination of two LFSRs. The construction bas attractive properties as simplicity (conceptual and implementation-wise), scalability (hardware and security), proven minimal security conditions (exponential period, exponential linear complexity, good statistical properties), and resistance to known attacks. The construction is suitable for practical implementation of efficient stream cipher cryptosystems.'


## Conclusion

This notebook goes into the details of Stream Ciphers, mainly focusing on the generation of keystreams using Pseudorandom Number Generators (PRNGs). Initially, we implemented a Linear Feedback Shift Register (LFSR), highlighting its characteristics and functionalities. Subsequently, we underscored its vulnerabilities, revealing its insecurity when employed as a standalone PRNG, being susceptible to the Berlekamp-Massey algorithm. Nonetheless, recognizing its utility in more intricate systems, we explored its potential in providing security when integrated into more complex PRNGs such as the Shrinking Generator. Eventually, the latter was implmeneted and used to successfully decrypt a given ciphertext.

## References

[1] [Stream Cipher](https://en.wikipedia.org/wiki/Stream_cipher)

[2] [Linear Feedback Shift Register](https://en.wikipedia.org/wiki/Linear-feedback_shift_register)

[3] [Berlekamp-Massey algorithm](https://en.wikipedia.org/wiki/Berlekamp%E2%80%93Massey_algorithm)

[4] [Shrinking Generator](https://link.springer.com/chapter/10.1007/3-540-48329-2_3)